In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우에서 사용 (리눅스에서는 'NanumGothic')
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 제대로 표시되게 설정

In [3]:
# 1. 데이터 불러오기
temp_1980 = './data/대한민국_여름기온_1980.xlsx'
temp_2023 = './data/대한민국_여름기온_2023.xlsx'

data_1980 = pd.read_excel(temp_1980)
data_2023 = pd.read_excel(temp_2023)

# 2. 데이터 확인
print("1980년 데이터:\n", data_1980.head())
print("2023년 데이터:\n", data_2023.head())

1980년 데이터:
        지점정보          경도         위도  평균기온(°C)  최저기온(°C)  최고기온(°C)
0    속초(90)  128.564730  38.250850      21.1      18.7      24.1
1  대관령(100)  128.758698  37.686924      17.0      13.6      21.2
2   춘천(101)  127.735700  37.902620      22.3      18.4      27.3
3   강릉(105)  128.890990  37.751470      22.0      19.0      26.0
4   서울(108)  126.965792  37.571411      22.4      19.1      26.6
2023년 데이터:
       지점정보         경도        위도  평균기온(°C)  최저기온(°C)  최고기온(°C)
0   속초(90)  128.56473  38.25085      24.1      21.0      28.1
1  북춘천(93)  127.75443  37.94738      24.8      20.6      30.1
2   철원(95)  127.30420  38.14787      24.1      19.9      29.1
3  동두천(98)  127.06070  37.90188      24.4      20.5      29.1
4   파주(99)  126.76648  37.88589      24.0      20.3      28.7


In [4]:
data_1980['지점정보'] = data_1980['지점정보'].str.replace(r'\(.*\)', '', regex=True) # (, () 사이의 모든 값, )을 모두 없는 값으로 바꿈
data_2023['지점정보'] = data_2023['지점정보'].str.replace(r'\(.*\)', '', regex=True)

max_1980 = data_1980[['지점정보', '경도', '위도', '최고기온(°C)']]
max_2023 = data_2023[['지점정보', '경도', '위도', '최고기온(°C)']]

print(data_1980.head(), '\n')
print(data_2023.head())

  지점정보          경도         위도  평균기온(°C)  최저기온(°C)  최고기온(°C)
0   속초  128.564730  38.250850      21.1      18.7      24.1
1  대관령  128.758698  37.686924      17.0      13.6      21.2
2   춘천  127.735700  37.902620      22.3      18.4      27.3
3   강릉  128.890990  37.751470      22.0      19.0      26.0
4   서울  126.965792  37.571411      22.4      19.1      26.6 

  지점정보         경도        위도  평균기온(°C)  최저기온(°C)  최고기온(°C)
0   속초  128.56473  38.25085      24.1      21.0      28.1
1  북춘천  127.75443  37.94738      24.8      20.6      30.1
2   철원  127.30420  38.14787      24.1      19.9      29.1
3  동두천  127.06070  37.90188      24.4      20.5      29.1
4   파주  126.76648  37.88589      24.0      20.3      28.7


In [49]:
max_1980

,지점정보,경도,위도,최고기온(°C)
0,속초,128.564730,38.250850,24.1
1,대관령,128.758698,37.686924,21.2
2,춘천,127.735700,37.902620,27.3
3,강릉,128.890990,37.751470,26.0
4,서울,126.965792,37.571411,26.6
...,...,...,...,...
58,합천,128.169940,35.565050,26.9
59,밀양,128.744120,35.491470,27.2
60,산청,127.879100,35.413000,26.9
61,거제,128.604590,34.888180,25.7


In [50]:
max_2023

,지점정보,경도,위도,최고기온(°C)
0,속초,128.56473,38.25085,28.1
1,북춘천,127.75443,37.94738,30.1
2,철원,127.30420,38.14787,29.1
3,동두천,127.06070,37.90188,29.1
4,파주,126.76648,37.88589,28.7
...,...,...,...,...
89,합천,128.16994,35.56505,30.5
90,밀양,128.74412,35.49147,31.2
91,산청,127.87910,35.41300,30.0
92,거제,128.60459,34.88818,28.4


In [92]:
heatmap_data_1980 = []
exclude_cities = ['성산포']#['성산포'] # 제외할 도시. 도시 중 같은 위도와 경도를 공유하는 경우들이 존재
for number, row in max_1980.iterrows(): # number = 인덱스, 숫자, row = 해당 행에 담긴 모든 데이터
    #print(row)
    #print(row[0])
    city = row[0]
    lat, lon, temp = row[1:4] # 위치 이름, 위도, 경도, 최대온도
    if city not in exclude_cities and temp > 27:
        heatmap_data_1980.append([lon, lat, temp])

C:\Users\USER\AppData\Local\Temp\ipykernel_21824\398875451.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city = row[0]


In [96]:
# 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start = 7)  # 대한민국 중심

# 히트맵 출력
HeatMap(heatmap_data_1980, radius = 40).add_to(m)
m

In [97]:
heatmap_data_2023 = []
for number, row in max_2023.iterrows(): # number = 인덱스, 숫자, row = 해당 행에 담긴 모든 데이터
    #print(row)
    #print(row[0])
    lat, lon, temp = row[1:4] # 위치 이름, 위도, 경도, 최대온도\
    if temp > 27:
        heatmap_data_2023.append([lon, lat, temp])


n = 0
for i in heatmap_data_2023:
    print(i)
    n += 1
    if n > 10:
        break

[38.25085, 128.56473, 28.1]
[37.94738, 127.75443, 30.1]
[38.14787, 127.3042, 29.1]
[37.90188, 127.0607, 29.1]
[37.88589, 126.76648, 28.7]
[37.90262, 127.7357, 29.8]
[37.80456, 128.85535, 28.0]
[37.75147, 128.89099, 29.6]
[37.50709, 129.12433, 27.3]
[37.57142, 126.9658, 29.6]
[37.47772, 126.6249, 27.8]


In [99]:
# 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start = 7)  # 대한민국 중심

# 히트맵 출력
HeatMap(heatmap_data_2023, radius = 40).add_to(m)
m